# From spreadsheet to Kobo — Part 2 (maybe 3?)
---

### Import whatever you need 👇

In [208]:
import io
import os
import copy
import requests
import uuid
from datetime import datetime
from time import sleep
from xml.etree import ElementTree as ET

import pandas as pd
import pytz

### Set up some helpful constants that we'll use below 🤘

In [4]:
# You can find this by navigating to https://kc.kobotoolbox.org/token
TOKEN = os.environ.get('KOBO_TOKEN')

In [257]:
KC_URL = 'https://kc.kobotoolbox.org'
KF_URL = 'https://kf.kobotoolbox.org'

ASSET_UID = 'aBebWB9QVH7hzW4PqzDJum'

ASSETS_URL = f'{KF_URL}/api/v2/assets/'
DATA_URL = f'{ASSETS_URL}{ASSET_UID}/data'
DEPLOYMENT_URL = '{}{}/deployment/'
XML_URL = f'{DATA_URL}.xml'
SUMISSION_URL = f'{KC_URL}/api/v1/submissions'
FORMS_URL = f'{KC_URL}/api/v1/forms'

HEADERS = {
    'Authorization': f'Token {TOKEN}'
}
PARAMS = {
    'format': 'json'
}

FILENAME = 'repeat_groups_export_2.xlsx'
REPEAT_GROUP_NAME = 'group_gp3qf47'

def _get_deployment_url(asset_uid):
    return DEPLOYMENT_URL.format(ASSETS_URL, asset_uid)

In [258]:
subs = pd.read_excel(FILENAME)
subs.head()

,start,end,What_is_your_name,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,_tags,_index
0,2021-03-31T12:15:42.060-07:00,2021-03-31T12:16:16.806-07:00,josh,90824294,62541aa7-3c76-4acd-817d-aea9c5df705b,2021-03-31T19:16:27,NaN,[],submitted_via_web,NaN,NaN,1
1,2021-03-31T12:16:16.829-07:00,2021-03-31T12:16:39.500-07:00,Naomi,90824366,4c23486d-9a76-4c4f-843f-2c00653e3048,2021-03-31T19:16:50,NaN,[],submitted_via_web,NaN,NaN,2
2,2021-03-31T12:16:39.513-07:00,2021-03-31T12:17:06.095-07:00,David,90824395,a68e72e8-b6b6-4929-9c1d-d61d50d400ba,2021-03-31T19:17:10,NaN,[],submitted_via_web,NaN,NaN,3


In [259]:
subs.columns.to_list()

['start',
 'end',
 'What_is_your_name',
 '_id',
 '_uuid',
 '_submission_time',
 '_validation_status',
 '_notes',
 '_status',
 '_submitted_by',
 '_tags',
 '_index']

In [260]:
subs_cols = ['What_is_your_name', '_uuid']
subs_filtered = subs[subs_cols]
subs_filtered.head()

,What_is_your_name,_uuid
0,josh,62541aa7-3c76-4acd-817d-aea9c5df705b
1,Naomi,4c23486d-9a76-4c4f-843f-2c00653e3048
2,David,a68e72e8-b6b6-4929-9c1d-d61d50d400ba


In [261]:
repeat_subs = pd.read_excel(FILENAME, sheet_name=REPEAT_GROUP_NAME)
repeat_subs.head()

,A_pizza_place_you_like,Your_favourite_toppings_there,Your_favourite_toppings_there/cheese,Your_favourite_toppings_there/pepperoni,Your_favourite_toppings_there/avo,_index,_parent_table_name,_parent_index,_submission__id,_submission__uuid,_submission__submission_time,_submission__validation_status,_submission__notes,_submission__status,_submission__submitted_by,_submission__tags
0,Place 1,cheese pepperoni avo,1,1,1,1,Repeat groups 2,1,90824294,62541aa7-3c76-4acd-817d-aea9c5df705b,2021-03-31T19:16:27,NaN,[],submitted_via_web,NaN,[]
1,Place 2,avo,0,0,1,2,Repeat groups 2,1,90824294,62541aa7-3c76-4acd-817d-aea9c5df705b,2021-03-31T19:16:27,NaN,[],submitted_via_web,NaN,[]
2,Place 1,pepperoni avo,0,1,1,3,Repeat groups 2,2,90824366,4c23486d-9a76-4c4f-843f-2c00653e3048,2021-03-31T19:16:50,NaN,[],submitted_via_web,NaN,[]
3,Place 3,cheese pepperoni avo,1,1,1,4,Repeat groups 2,2,90824366,4c23486d-9a76-4c4f-843f-2c00653e3048,2021-03-31T19:16:50,NaN,[],submitted_via_web,NaN,[]
4,Place 1,cheese pepperoni avo,1,1,1,5,Repeat groups 2,3,90824395,a68e72e8-b6b6-4929-9c1d-d61d50d400ba,2021-03-31T19:17:10,NaN,[],submitted_via_web,NaN,[]


# Data cleaning

In [262]:
repeat_subs_cols = ['A_pizza_place_you_like', 'Your_favourite_toppings_there', '_submission__uuid']
repeat_subs_filtered = repeat_subs[repeat_subs_cols]
repeat_subs_filtered.head()

,A_pizza_place_you_like,Your_favourite_toppings_there,_submission__uuid
0,Place 1,cheese pepperoni avo,62541aa7-3c76-4acd-817d-aea9c5df705b
1,Place 2,avo,62541aa7-3c76-4acd-817d-aea9c5df705b
2,Place 1,pepperoni avo,4c23486d-9a76-4c4f-843f-2c00653e3048
3,Place 3,cheese pepperoni avo,4c23486d-9a76-4c4f-843f-2c00653e3048
4,Place 1,cheese pepperoni avo,a68e72e8-b6b6-4929-9c1d-d61d50d400ba


In [264]:
USER = 'josh'
user_uid = subs_filtered[subs_filtered['What_is_your_name'] == USER]['_uuid'].values[0]
user_uid

'62541aa7-3c76-4acd-817d-aea9c5df705b'

In [265]:
repeat_subs_for_user = repeat_subs_filtered[repeat_subs_filtered['_submission__uuid'] == user_uid]
repeat_subs_for_user

,A_pizza_place_you_like,Your_favourite_toppings_there,_submission__uuid
0,Place 1,cheese pepperoni avo,62541aa7-3c76-4acd-817d-aea9c5df705b
1,Place 2,avo,62541aa7-3c76-4acd-817d-aea9c5df705b


In [266]:
def remove_choice(string_choices, choice):
    return ' '.join([c for c in string_choices.split() if c != choice])

In [267]:
def map_place_names(place):
    places = {
        'Place 1': 'Limoncello',
        'Place 2': 'The Backyard',
        'Place 3': 'Basilico'
    }
    new_place = places.get(place)
    return new_place if new_place is not None else place

In [272]:
uuids_for_updating = [
    '62541aa7-3c76-4acd-817d-aea9c5df705b', 
    '4c23486d-9a76-4c4f-843f-2c00653e3048',
    'a68e72e8-b6b6-4929-9c1d-d61d50d400ba'
]

In [273]:
for i, row in repeat_subs_filtered.iterrows():
    if row['_submission__uuid'] in uuids_for_updating:
        row['Your_favourite_toppings_there'] = remove_choice(row['Your_favourite_toppings_there'], 'pepperoni')
    row['A_pizza_place_you_like'] = map_place_names(row['A_pizza_place_you_like'])

In [274]:
repeat_subs_filtered

,A_pizza_place_you_like,Your_favourite_toppings_there,_submission__uuid
0,Limoncello,cheese avo,62541aa7-3c76-4acd-817d-aea9c5df705b
1,The Backyard,avo,62541aa7-3c76-4acd-817d-aea9c5df705b
2,Limoncello,avo,4c23486d-9a76-4c4f-843f-2c00653e3048
3,Basilico,cheese avo,4c23486d-9a76-4c4f-843f-2c00653e3048
4,Limoncello,cheese avo,a68e72e8-b6b6-4929-9c1d-d61d50d400ba
5,The Backyard,cheese avo,a68e72e8-b6b6-4929-9c1d-d61d50d400ba
6,Basilico,avo,a68e72e8-b6b6-4929-9c1d-d61d50d400ba


In [275]:
updated_subs = []
for i, subs_row in subs_filtered.iterrows():
    sub = {
        'What_is_your_name': subs_row['What_is_your_name'],
        '_uuid': subs_row['_uuid']
    }
    
    df_repeats = repeat_subs_filtered[repeat_subs_filtered['_submission__uuid'] == sub['_uuid']]
    repeat_group = []
    for j, repeats_row in df_repeats.iterrows():
        repeat_group.append({
            'A_pizza_place_you_like': repeats_row['A_pizza_place_you_like'],
            'Your_favourite_toppings_there': repeats_row['Your_favourite_toppings_there']
        })
    sub[REPEAT_GROUP_NAME] = repeat_group
    updated_subs.append(sub)

In [276]:
updated_subs

[{'What_is_your_name': 'josh',
  '_uuid': '62541aa7-3c76-4acd-817d-aea9c5df705b',
  'group_gp3qf47': [{'A_pizza_place_you_like': 'Limoncello',
    'Your_favourite_toppings_there': 'cheese avo'},
   {'A_pizza_place_you_like': 'The Backyard',
    'Your_favourite_toppings_there': 'avo'}]},
 {'What_is_your_name': 'Naomi',
  '_uuid': '4c23486d-9a76-4c4f-843f-2c00653e3048',
  'group_gp3qf47': [{'A_pizza_place_you_like': 'Limoncello',
    'Your_favourite_toppings_there': 'avo'},
   {'A_pizza_place_you_like': 'Basilico',
    'Your_favourite_toppings_there': 'cheese avo'}]},
 {'What_is_your_name': 'David',
  '_uuid': 'a68e72e8-b6b6-4929-9c1d-d61d50d400ba',
  'group_gp3qf47': [{'A_pizza_place_you_like': 'Limoncello',
    'Your_favourite_toppings_there': 'cheese avo'},
   {'A_pizza_place_you_like': 'The Backyard',
    'Your_favourite_toppings_there': 'cheese avo'},
   {'A_pizza_place_you_like': 'Basilico',
    'Your_favourite_toppings_there': 'avo'}]}]

# Get submission XML

In [311]:
res = requests.get(url=XML_URL, headers=HEADERS, params=PARAMS)

In [312]:
res.status_code

200

In [313]:
parsed_xml = ET.fromstring(res.text)
e = parsed_xml.find(f'results/{ASSET_UID}')
print(ET.tostring(e).decode())

<aBebWB9QVH7hzW4PqzDJum id="aBebWB9QVH7hzW4PqzDJum" version="1 (2021-03-31 19:14:56)">
          <formhub>
            <uuid>68349f0819ab4847920631542c1f97ec</uuid>
          </formhub>
          <start>2021-03-31T12:15:42.060-07:00</start>
          <end>2021-03-31T12:16:16.806-07:00</end>
          <What_is_your_name>josh</What_is_your_name>
          <group_gp3qf47>
            <A_pizza_place_you_like>Place 1</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese pepperoni avo</Your_favourite_toppings_there>
          </group_gp3qf47><group_gp3qf47>
            <A_pizza_place_you_like>Place 2</A_pizza_place_you_like>
            <Your_favourite_toppings_there>avo</Your_favourite_toppings_there>
          </group_gp3qf47>
          <__version__>vfZucefRu2xp4fShpCCrnC</__version__>
          <meta>
            <instanceID>uuid:62541aa7-3c76-4acd-817d-aea9c5df705b</instanceID>
          </meta>
        </aBebWB9QVH7hzW4PqzDJum>


In [318]:
e.findall('group_gp3qf47')

[<Element 'group_gp3qf47' at 0x7fedbbc5ad60>,
 <Element 'group_gp3qf47' at 0x7fedbbc5a430>]

In [319]:
all_xml_subs = parsed_xml.findall(f'results/{ASSET_UID}')
all_xml_subs

[<Element 'aBebWB9QVH7hzW4PqzDJum' at 0x7fedbbc5aac0>,
 <Element 'aBebWB9QVH7hzW4PqzDJum' at 0x7fedbbc7b220>,
 <Element 'aBebWB9QVH7hzW4PqzDJum' at 0x7fedbbc7b7c0>]

In [320]:
def get_xml_for_submission_uid(xml, submission_uid):
    return [x for x in xml if x.find('meta/instanceID').text == f'uuid:{submission_uid}'][0]

In [321]:
tmp_el = get_xml_for_submission_uid(all_xml_subs, '62541aa7-3c76-4acd-817d-aea9c5df705b')
tmp_el

<Element 'aBebWB9QVH7hzW4PqzDJum' at 0x7fedbbc5aac0>

In [285]:
tmp_el.attrib

{'id': 'aBebWB9QVH7hzW4PqzDJum', 'version': '1 (2021-03-31 19:14:56)'}

In [286]:
print(ET.tostring(tmp_el).decode())

<aBebWB9QVH7hzW4PqzDJum id="aBebWB9QVH7hzW4PqzDJum" version="1 (2021-03-31 19:14:56)">
          <formhub>
            <uuid>68349f0819ab4847920631542c1f97ec</uuid>
          </formhub>
          <start>2021-03-31T12:16:39.513-07:00</start>
          <end>2021-03-31T12:17:06.095-07:00</end>
          <What_is_your_name>David</What_is_your_name>
          <group_gp3qf47>
            <A_pizza_place_you_like>Place 1</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese pepperoni avo</Your_favourite_toppings_there>
          </group_gp3qf47><group_gp3qf47>
            <A_pizza_place_you_like>Place 2</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese pepperoni avo</Your_favourite_toppings_there>
          </group_gp3qf47><group_gp3qf47>
            <A_pizza_place_you_like>Place 3</A_pizza_place_you_like>
            <Your_favourite_toppings_there>pepperoni avo</Your_favourite_toppings_there>
          </group_gp3qf47>
          <__version__>vfZuc

In [169]:
def submit_data(xml_sub: bytes, _uuid: str) -> str:
    """
    Send the XML to kobo!
    """
    file_tuple = (_uuid, io.BytesIO(xml_sub))
    files = {'xml_submission_file': file_tuple}
    res = requests.Request(
        method='POST', url=SUMISSION_URL, files=files, headers=HEADERS
    )
    session = requests.Session()
    res = session.send(res.prepare())
    return res.status_code

def format_openrosa_datetime() -> str:
    """
    This is required to get the correct datetime formatting
    """
    return datetime.now(tz=pytz.UTC).isoformat('T', 'milliseconds')

In [287]:
def update_element_value(e: ET.Element, name: str, value: str) -> None:
    """
    Get or create a node and give it a value, even if nested within a group
    """
    el = e.find(name)
    if el is None:
        # this only works for questions nested within a single group
        # and will need to be extended to handle deeply nested groups
        # refer here for how this is implimented in KPI:
        # https://github.com/kobotoolbox/kpi/blob/27cff694f9a63fd091ba8b803ac925f77b8f513b/kpi/deployment_backends/kobocat_backend.py#L827-L857
        if '/' in name:
            root, node = name.split('/')
            el = ET.SubElement(e.find(root), node)
        else:
            el = ET.SubElement(e, name)
    el.text = value
    
def update_list_of_element_values(e: ET.Element, name: str, items: str) -> None:
    """
    Update a list of nodes within a group -- will fail if the item doesn't exist
    on the XML tree so extend appropriately
    """
    els = e.findall(name)
    for el, item in zip(els, items):
        for k, v in item.items():
            el.find(k).text = v
            
def update_root_element_tag_and_attrib(e: ET.Element, tag: str, attrib: dict) -> None:
    e.tag = tag
    e.attrib = attrib

In [316]:
def create_submissions(subs: list, clone_data=None) -> list:
    """
    Take a bunch of submissions and send them off
    
    If wanting to use a cloned project, the dict must contain the following data
    clone_data = {
        'clone_asset_uid': '123456',
        'version': '1 (234567)',
        '__version__': 'sgsgsgsdg',
        'formhub_uuid': '123456ytfdfaw'
    }
    
    """
    
    all_subs = []
    for sub in subs:
        sub = copy.deepcopy(sub)
        sub_uuid = sub.pop('_uuid')
        parsed_xml = get_xml_for_submission_uid(all_xml_subs, sub_uuid)
        
        _now = format_openrosa_datetime()
        _uuid = str(uuid.uuid4())
        
        for k, v in sub.items():
            if isinstance(v, list):
                update_list_of_element_values(parsed_xml, k, v)
            else:
                update_element_value(parsed_xml, k, v)
        
        # We have to create a meta/deprecatedID node with the value of the old instanceID and then 
        # update the instanceID for the submission to be updated correctly on the server -- ONLY IF 
        # NOT USING A CLONED FORM
        if clone_data is None:
            update_element_value(parsed_xml, 'meta/deprecatedID', parsed_xml.find('meta/instanceID').text)
        update_element_value(parsed_xml, 'meta/instanceID', f'uuid:{_uuid}')
        
        # Updating the `start` and `end` times is not really necessary, but 
        # probably something you'd want to do
        update_element_value(parsed_xml, 'start', _now)
        update_element_value(parsed_xml, 'end', _now)
        
        if clone_data is not None:
            new_attrib = {
                'id': clone_data['clone_asset_uid'],
                'version': clone_data['version']
            }
            update_root_element_tag_and_attrib(parsed_xml, clone_data['clone_asset_uid'], new_attrib)
            update_element_value(parsed_xml, '__version__', clone_data['__version__'])
            update_element_value(parsed_xml, 'formhub/uuid', clone_data['formhub_uuid'])
        
        all_subs.append(submit_data(ET.tostring(parsed_xml), _uuid))
        
        # If you are submitting a large amount of data, please be mindful that it can
        # overwhelm the servers if sent in a short span of time. Letting it sleep for
        # for a short stint between each upload will be much appreciated
        sleep(0.2)
        
    return all_subs

In [314]:
updated_subs

[{'What_is_your_name': 'josh',
  '_uuid': '62541aa7-3c76-4acd-817d-aea9c5df705b',
  'group_gp3qf47': [{'A_pizza_place_you_like': 'Limoncello',
    'Your_favourite_toppings_there': 'cheese avo'},
   {'A_pizza_place_you_like': 'The Backyard',
    'Your_favourite_toppings_there': 'avo'}]},
 {'What_is_your_name': 'Naomi',
  '_uuid': '4c23486d-9a76-4c4f-843f-2c00653e3048',
  'group_gp3qf47': [{'A_pizza_place_you_like': 'Limoncello',
    'Your_favourite_toppings_there': 'avo'},
   {'A_pizza_place_you_like': 'Basilico',
    'Your_favourite_toppings_there': 'cheese avo'}]},
 {'What_is_your_name': 'David',
  '_uuid': 'a68e72e8-b6b6-4929-9c1d-d61d50d400ba',
  'group_gp3qf47': [{'A_pizza_place_you_like': 'Limoncello',
    'Your_favourite_toppings_there': 'cheese avo'},
   {'A_pizza_place_you_like': 'The Backyard',
    'Your_favourite_toppings_there': 'cheese avo'},
   {'A_pizza_place_you_like': 'Basilico',
    'Your_favourite_toppings_there': 'avo'}]}]

# Approach 1: Send modified data to cloned form

Steps:
- clone
- deploy
- get deployment data from kc and kpi
- hack hack

## Clone form

In [288]:
data = {
    'clone_from': ASSET_UID,
    'name': f'Test clone of Repeat Groups 2from Jupyter {uuid.uuid4()}'
}
res = requests.post(url=ASSETS_URL, headers=HEADERS, params=PARAMS, data=data)

In [289]:
res.status_code

201

In [290]:
cloned_survey = res.json()

In [291]:
cloned_asset_uid = cloned_survey['uid']
cloned_asset_uid

'azNBzazvrjaqmmgF2H7gF5'

## Deploy new form

In [292]:
data = {
    'active': 'true'
}
res = requests.post(url=_get_deployment_url(cloned_asset_uid), headers=HEADERS, params=PARAMS, data=data)

In [293]:
res.status_code

200

## Get new from uuid

In [294]:
res = requests.get(url=FORMS_URL, headers=HEADERS, params=PARAMS)

In [295]:
res.status_code

200

In [296]:
all_forms = res.json()

In [297]:
latest_form = [f for f in all_forms if f['id_string'] == cloned_asset_uid][0]
latest_form

{'url': 'https://kc.kobotoolbox.org/api/v1/forms/647717?format=json',
 'formid': 647717,
 'metadata': [],
 'owner': 'joshuaberetta',
 'public': False,
 'public_data': False,
 'require_auth': False,
 'tags': [],
 'title': 'Test clone of Repeat Groups 2from Jupyter 7c14d703-783a-4b4e-b1b7-49910f94275c',
 'users': [{'user': 'joshuaberetta',
   'permissions': ['add_datadictionary',
    'add_xform',
    'change_datadictionary',
    'change_xform',
    'delete_data_xform',
    'delete_datadictionary',
    'delete_xform',
    'move_xform',
    'report_xform',
    'transfer_xform',
    'validate_xform',
    'view_xform']}],
 'hash': 'md5:b5c6e7895279f7dd628a3248d0dc8c6e',
 'has_kpi_hooks': False,
 'description': 'Test clone of Repeat Groups 2from Jupyter 7c14d703-783a-4b4e-b1b7-49910f94275c',
 'downloadable': True,
 'allows_sms': False,
 'encrypted': False,
 'sms_id_string': 'azNBzazvrjaqmmgF2H7gF5',
 'id_string': 'azNBzazvrjaqmmgF2H7gF5',
 'date_created': '2021-03-31T19:22:13.827490Z',
 'date

In [299]:
formhub_uuid = latest_form['uuid']
formhub_uuid

'30f53e5a4bc0425fb5bb27819c14ba44'

## Get deployent info

There are a few things we need to update on each submission's XML:
- The root `tag` and attributes of `id` and `version`
- the value of `formhub/uuid` with the new uuid
- value of `__version__`

In [300]:
res = requests.get(f'{ASSETS_URL}{cloned_asset_uid}', headers=HEADERS, params=PARAMS)

In [301]:
res.status_code

200

In [302]:
deployed_versions = res.json()['deployed_versions']
deployed_versions

{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'uid': 'vuoUnu6ZRrYwbxGz48QLSZ',
   'url': 'https://kf.kobotoolbox.org/api/v2/assets/azNBzazvrjaqmmgF2H7gF5/versions/vuoUnu6ZRrYwbxGz48QLSZ/?format=json',
   'content_hash': 'b47ac0203f253524aaada90e13575b5a0041ee7b',
   'date_deployed': '2021-03-31T19:22:02.626863Z',
   'date_modified': '2021-03-31 19:22:02.626863+00:00'}]}

In [303]:
def format_date_string(date_str):
    """
    goal: "1 (2021-03-29 19:40:28)"
    """
    date, time = date_str.split('T')
    return f"{date} {time.split('.')[0]}"

In [304]:
def get_info_from_deployed_versions(deployed_versions):
    count = deployed_versions['count']
    
    latest_deployment = deployed_versions['results'][0]
    date = latest_deployment['date_deployed']
    version = latest_deployment['uid']
    
    return version, f'{count} ({format_date_string(date)})'

In [305]:
get_info_from_deployed_versions(deployed_versions)

('vuoUnu6ZRrYwbxGz48QLSZ', '1 (2021-03-31 19:22:02)')

In [306]:
_v_, v = get_info_from_deployed_versions(deployed_versions)
clone_data = {
        'clone_asset_uid': cloned_asset_uid,
        'version': v,
        '__version__': _v_,
        'formhub_uuid': formhub_uuid
    }

In [322]:
responses = create_submissions(updated_subs, clone_data)

62541aa7-3c76-4acd-817d-aea9c5df705b
4c23486d-9a76-4c4f-843f-2c00653e3048
a68e72e8-b6b6-4929-9c1d-d61d50d400ba


In [323]:
all(res == 201 for res in responses)

[201, 201, 201]

In [ ]:
pd.Series(resonses).value_counts()

# Approach 2: Send modified data back to existing form
---

### Word of caution: this will overwrite your existing submissions, so be very sure that you know what you're doing otherwise you will corrupt your data.

- update the `start` and `end` dates
- move `instanceID` to `deprecatedID`
- create new `instanceID`
- send off

In [157]:
responses = create_submissions(updated_subs)

In [158]:
all(res == 201 for res in responses)

[201, 201, 201]

In [127]:
pd.Series(resonses).value_counts()

<aN7gj3BsHV9pNPhnDXrKE3 id="aN7gj3BsHV9pNPhnDXrKE3" version="1 (2021-03-29 19:40:28)">
          <formhub>
            <uuid>0b21fc06052142e4bd7594e20a1e39fb</uuid>
          </formhub>
          <start>2021-03-31T17:22:36.876+00:00</start>
          <end>2021-03-31T17:22:36.876+00:00</end>
          <What_is_your_name>Josh</What_is_your_name>
          <group_gp3qf47>
            <A_pizza_place_you_like>Limoncello</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese</Your_favourite_toppings_there>
          </group_gp3qf47><group_gp3qf47>
            <A_pizza_place_you_like>The Backyard</A_pizza_place_you_like>
            <Your_favourite_toppings_there>avo</Your_favourite_toppings_there>
          </group_gp3qf47><group_gp3qf47>
            <A_pizza_place_you_like>Basilico</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese avo</Your_favourite_toppings_there>
          </group_gp3qf47>
          <__version__>vuYnF2nxowu7CQN8LEhTgW</__versi